In [26]:
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
from dateutil.relativedelta import relativedelta

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline, get_baseline_tdx,convert_daily_return_to_pyfolio_ts

In [2]:
from lutils.stock import LTdxHq

In [3]:
ddf = pd.read_pickle('d:/ddf.pkl').dropna()

In [19]:
ddf

,open,close,high,low,volume,amount,tic,date,day
0,9.23,8.71,9.25,8.63,56349700.0,6.603760e+08,000001,2016-01-04,0
0,10.00,9.04,10.00,9.04,32340600.0,3.022400e+08,000005,2016-01-04,0
0,9.67,8.76,9.72,8.74,23285272.0,2.546398e+08,000006,2016-01-04,0
0,11.79,10.53,11.79,10.52,14039993.0,1.570146e+08,000008,2016-01-04,0
0,10.89,9.80,10.89,9.80,67253104.0,1.130234e+09,000009,2016-01-04,0
...,...,...,...,...,...,...,...,...,...
1471,26.33,26.80,27.60,26.10,2115253.0,5.725555e+07,688698,2022-01-18,1
1471,14.34,13.88,14.37,13.75,1527267.0,2.138187e+07,688701,2022-01-18,1
1471,13.94,13.38,14.06,13.38,3075937.0,4.200164e+07,688722,2022-01-18,1
1471,41.68,42.29,42.72,41.35,3164833.0,1.336842e+08,688819,2022-01-18,1


In [4]:
ddf = ddf.sort_values(['date','tic'], ignore_index=True)
ddf.index = ddf.date.factorize()[0]

In [32]:
start_day = datetime.datetime.strptime('2021-07-01', '%Y-%m-%d')

In [33]:
sc = []
for i in range(6):
    print(start_day)
    dl = ddf[ddf.date < start_day].loc[-252:,:]
    dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
    corr = dd.cov().corr()
    sc.append(list(corr.sum().sort_values().index.values[:6]))
    
    start_day = start_day + relativedelta(months=+1)

2021-07-01 00:00:00
2021-08-01 00:00:00
2021-09-01 00:00:00
2021-10-01 00:00:00
2021-11-01 00:00:00
2021-12-01 00:00:00


In [34]:
sc

[['000001', '600569', '600566', '600565', '600560', '600558'],
 ['000001', '600537', '600546', '600549', '600551', '600552'],
 ['000001', '600535', '600537', '600546', '600549', '600551'],
 ['000001', '600535', '600537', '600546', '600549', '600551'],
 ['000001', '600531', '600535', '600537', '600546', '600549'],
 ['603360', '600062', '600300', '002436', '603331', '300735']]

In [5]:
# dl = ddf.loc[-252:,:]
# dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
# corr = dd.cov().corr()
# # corr.style.background_gradient(cmap='coolwarm')

# # c1 = corr.abs().unstack().sort_values(ascending = True)
# c1 = corr.unstack().sort_values(ascending = True)

In [6]:
# corr.style.background_gradient(cmap='coolwarm')

In [7]:
# corr.sum().sort_values()

In [8]:
# stock_codes = list(corr.sum().sort_values().index.values[:6])

In [9]:
# stock_codes = ['000921', '002032', '300406', '603789']
# stock_codes = ['000609', '000921', '001965', '002372']

In [36]:
ltdxhq = LTdxHq()

ddfs = []
for stock_codes in sc:
    indexs = None
    dfs = []
    for code in stock_codes:
        df = ltdxhq.get_k_data_daily(code, start='2018-01-01') # 2014-01-01

        if indexs is None:
            indexs = df.index
        else:
            indexs = indexs.union(df.index)

    #     df = df.assign(date = df.index)
    #     df = df.assign(day = df.index.weekday)
    #     df.date = df.date.dt.strftime('%Y-%m-%d')
        df = df.assign(tic = code)
    #     df.index = range(df.shape[0])

        dfs.append(df)
        print('----------- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))

    for i, df in enumerate(dfs):
        df = df.reindex(indexs)
        df = df.assign(date = df.index)
        df = df.assign(day = df.index.weekday)
        df.index = range(df.shape[0])

        dfs[i] = df.ffill()

    df = pd.concat(dfs)
    ddfs.append(df)
    # df.index = range(df.shape[0])

ltdxhq.close()

----------- over 000001 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600569 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600566 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600565 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600560 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600558 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 000001 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600537 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600546 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600549 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600551 min: 2018-01-02 00:00:00 max: 2022-01-18 00:00:00 -----------
----------- over 600552 min: 2018-01-02 00:00:00 max: 

In [11]:
fe = FeatureEngineer(use_technical_indicator=True,
                     use_turbulence=False,
                     user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [12]:
# add covariance matrix as states
df=df.sort_values(['date','tic'], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i,:]
    price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list': cov_list,'return_list': return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

In [13]:
df['daily_variance'] = (df.high-df.low) / df.close

In [14]:
dl = df.loc[-252:, :]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
dd.cov().corr().style.background_gradient(cmap='coolwarm')

tic,000609,000921,001965,002372
tic,,,,
000609,1.000000,-0.232511,-0.228589,-0.296210
000921,-0.232511,1.000000,-0.395016,0.104416
001965,-0.228589,-0.395016,1.000000,-0.767609
002372,-0.296210,0.104416,-0.767609,1.000000


In [15]:
indexes = df.date.unique()
start_date = indexes[-126:]

In [37]:
indexes

array(['2019-01-15T00:00:00.000000000', '2019-01-16T00:00:00.000000000',
       '2019-01-17T00:00:00.000000000', '2019-01-18T00:00:00.000000000',
       '2019-01-21T00:00:00.000000000', '2019-01-22T00:00:00.000000000',
       '2019-01-23T00:00:00.000000000', '2019-01-24T00:00:00.000000000',
       '2019-01-25T00:00:00.000000000', '2019-01-28T00:00:00.000000000',
       '2019-01-29T00:00:00.000000000', '2019-01-30T00:00:00.000000000',
       '2019-01-31T00:00:00.000000000', '2019-02-01T00:00:00.000000000',
       '2019-02-11T00:00:00.000000000', '2019-02-12T00:00:00.000000000',
       '2019-02-13T00:00:00.000000000', '2019-02-14T00:00:00.000000000',
       '2019-02-15T00:00:00.000000000', '2019-02-18T00:00:00.000000000',
       '2019-02-19T00:00:00.000000000', '2019-02-20T00:00:00.000000000',
       '2019-02-21T00:00:00.000000000', '2019-02-22T00:00:00.000000000',
       '2019-02-25T00:00:00.000000000', '2019-02-26T00:00:00.000000000',
       '2019-02-27T00:00:00.000000000', '2019-02-28

In [16]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                 df,
                 stock_dim,
                 hmax,
                 initial_amount,
                 transaction_cost_pct,
                 reward_scaling,
                 state_space,
                 action_space,
                 tech_indicator_list,
                 turbulence_threshold=None,
                 lookback=252,
                 day=0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
#         self.reward = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1,shape=(self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1 / self.stock_dim] * self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
#             plt.plot(df.daily_return.cumsum(),'r')
#             plt.savefig('results/cumulative_reward.png')
#             plt.close()
            
#             plt.plot(self.portfolio_return_memory,'r')
#             plt.savefig('results/rewards.png')
#             plt.close()

#             print("=================================")
#             print("begin_total_asset:{}".format(self.asset_memory[0]))
#             print("end_total_asset:{}".format(self.portfolio_value))

#             df_daily_return = pd.DataFrame(self.portfolio_return_memory)
#             df_daily_return.columns = ['daily_return']
#             if df_daily_return['daily_return'].std() != 0:
#                 sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / df_daily_return['daily_return'].std()
#                 print("Sharpe: ", sharpe)
#             print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list], axis=0)
#             portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            new_portfolio_value = self.portfolio_value * (1 + log_portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [17]:
initial_amount = 100000

# model_ppo = None

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}

for i, date in enumerate(start_date[:-2]):
    train = data_split(df, '2019-01-01', date)
    
    stock_dimension = len(train.tic.unique())
    state_space = stock_dimension
    
    tech_indicator_list = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'daily_variance', 'dx_30', 'close_30_sma', 'close_60_sma']
    env_kwargs = {
        "hmax": 100, 
        "initial_amount": initial_amount, 
        "transaction_cost_pct": 0, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": tech_indicator_list, # config.TECHNICAL_INDICATORS_LIST, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-1
    }
    
    e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
    env_train, _ = e_train_gym.get_sb_env()

    agent = DRLAgent(env = env_train)
    
    if model_ppo is None:
        model_ppo = agent.get_model("ppo", verbose=0, model_kwargs = PPO_PARAMS)
    
    trained_ppo = agent.train_model(model=model_ppo, tb_log_name='ppo', total_timesteps=40000)
    
    ################################################
    trade = data_split(df, date, start_date[i+2])
    
    e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)
    
    df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=trained_ppo, environment=e_trade_gym)
    time_ind = pd.Series(df_daily_return_ppo.date)
    ppo_cumpod =(df_daily_return_ppo.daily_return + 1).cumprod() - 1
    DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)
    
    initial_amount = initial_amount * (ppo_cumpod[1] + 1)
    
#     print(ppo_cumpod)
#     print(ppo_cumpod)
    print("Over Date %s: %s, %s" % (date, initial_amount, ppo_cumpod[1]))

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-07-15T00:00:00.000000000: 101141.39566915353, 0.01141395669153522
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-07-16T00:00:00.000000000: 102868.7872509629, 0.017078977112990223
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-07-19T00:00:00.000000000: 101379.22147029659, -0.014480250234041425
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-07-20T00:00:00.000000000: 101165.28832196616, -0.0021102267824487653
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-07-21T00:00:00.000000000: 100025.66427166403, -0.011264971110201194
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-07-22T00:00:00.000000000: 98592.1

hit end!
Over Date 2021-09-23T00:00:00.000000000: 104525.88486504645, -0.022280966117688816
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-09-24T00:00:00.000000000: 101645.66958936867, -0.027555043225861553
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-09-27T00:00:00.000000000: 102087.83637660537, 0.00435007992984815
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-09-28T00:00:00.000000000: 100438.16717358124, -0.01615931203535781
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-09-29T00:00:00.000000000: 101339.77778053774, 0.008976772797917532
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
hit end!
Over Date 2021-09-30T00:00:00.000000000: 103810.76034293996, 0.024383145656322602
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_r

KeyboardInterrupt: 

In [ ]:
# date

In [ ]:
# trade = data_split(df, '2021-07-20', '2021-08-20')
# e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)
# df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=model_ppo, environment=e_trade_gym)

In [ ]:
# time_ind = pd.Series(df_daily_return_ppo.date)
# ppo_cumpod =(df_daily_return_ppo.daily_return + 1).cumprod() - 1
# DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)

In [ ]:
# ppo_cumpod